# [CDAF] Atividade 4

## Nome e matrícula
Nome: Antônio Caetano Neves Neto

Matrícula: 2022043698

## Referências
- [1] https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
- [2] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.ExpectedThreat.html#socceraction.xthreat.ExpectedThreat
- [3] https://socceraction.readthedocs.io/en/latest/api/generated/socceraction.xthreat.get_successful_move_actions.html#socceraction.xthreat.get_successful_move_actions
- [4] https://socceraction.readthedocs.io/en/latest/documentation/valuing_actions/xT.html

In [1]:
# Importando bibliotecas
from tqdm import tqdm
import numpy as np
import pandas as pd
import socceraction.spadl as spd
from socceraction import xthreat as xt

### LaLiga  p/ SPADL com pré-processamentos

In [2]:
# carregando os eventos
path ="events_Spain.json"
events = pd.read_json(path_or_buf=path)

In [3]:
# pré processamento em colunas da tabela de eventos para facilitar a conversão p/ SPADL
events = events.rename(columns={'id': 'event_id', 'eventId': 'type_id', 'subEventId': 'subtype_id',
                                'teamId': 'team_id', 'playerId': 'player_id', 'matchId': 'game_id'})
events['milliseconds'] = events['eventSec'] * 1000
events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})

C:\Users\tonim\AppData\Local\Temp\ipykernel_23704\3735303047.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  events['period_id'] = events['matchPeriod'].replace({'1H': 1, '2H': 2})


In [4]:
# carregando as partidas, pois vamos saber quais times jogam em casa e fora p/ usar como parametro do SPADL
path = "matches_Spain.json"
matches = pd.read_json(path_or_buf=path)

In [6]:
# as informações dos times de cada partida estão em um dicionário dentro da coluna 'teamsData', então vamos separar essas informações
team_matches = []
for i in tqdm(range(len(matches))):
    match = pd.DataFrame(matches.loc[i, 'teamsData']).T
    match['matchId'] = matches.loc[i, 'wyId']
    team_matches.append(match)
team_matches = pd.concat(team_matches).reset_index(drop=True)

100%|███████████████████████████████████████████████████████████████████████████████| 380/380 [00:00<00:00, 719.33it/s]


In [7]:
# fazendo a conversão p/ SPADL, padronizando a direção de jogo da esquerda p/ a direita e adicionando os nomes dos tipos de ações
spadl = []
game_ids = events.game_id.unique().tolist()
for g in tqdm(game_ids):
    match_events = events.loc[events.game_id == g]
    match_home_id = team_matches.loc[(team_matches.matchId == g) & (team_matches.side == 'home'), 'teamId'].values[0]
    match_actions = spd.wyscout.convert_to_actions(events=match_events, home_team_id=match_home_id)
    match_actions = spd.play_left_to_right(actions=match_actions, home_team_id=match_home_id)
    match_actions = spd.add_names(match_actions)
    spadl.append(match_actions)
spadl = pd.concat(spadl).reset_index(drop=True)

100%|████████████████████████████████████████████████████████████████████████████████| 380/380 [05:14<00:00,  1.21it/s]


In [8]:
# adicionando o nome dos jogadores
path = "players.json"
players = pd.read_json(path_or_buf=path)
players['player_name'] = players['firstName'] + ' ' + players['lastName']
players = players[['wyId', 'player_name']].rename(columns={'wyId': 'player_id'})
spadl = spadl.merge(players, on='player_id', how='left')

## Questão 1
- Crie um dataframe "shots" à partir do dataframe "spadl", contendo apenas os chutes.
- Crie 4 colunas no dataframe "shots" a serem usadas como features de um modelo de xG.
- Justifique a escolha das features.

In [27]:
shots = spadl.query("type_name == 'shot'")

In [35]:
# Distância ao centro do gol
# Campo vai de (0, 105) a (0, 68), então o centro dos gols é (0, 34) e (105, 34)
# É pego a distância mínima entre os dois gols para fins de facilitação (chutes antes do meio de campo são mais raros)

distance_right = np.sqrt(((shots["start_x"])**2 + (shots["start_y"] - 34)**2))
distance_left = np.sqrt(((shots["start_x"] - 105)**2 + (shots["start_y"] - 34)**2))

distance_to_goal_center = pd.concat([distance_right, distance_left], axis=1).min(axis=1)

In [47]:
# Ângulo ao centro do gol
# Pega o gol mais próximo com justificativa igual antes
distance_x = pd.concat([shots["start_x"], 105-shots["start_x"]], axis = 1).min(axis=1)
distance_y = np.abs(shots["start_y"] - 34)

angles_to_goal_center = np.arctan(distance_y / distance_x) * 180 / np.pi

In [48]:
# Parte do corpo usada para o chute
bodypart = shots["bodypart_id"]

In [66]:
# Time da casa ou não

# team_matches.set_index("matchId", drop=True, inplace=True)

home_or_away = []

for i, row in shots.iterrows():
    game_id = row["game_id"]
    team_id = row["team_id"]

    home_or_away.append(
        team_matches.loc[game_id, ["side", "teamId"]].query(f"teamId == {team_id}").iloc[0]["side"] == "home"
    )

In [102]:
features = pd.DataFrame({
    "distance_to_goal": distance_to_goal_center.tolist(),
    "angle_to_goal": angles_to_goal_center.tolist(),
    "bodypart": bodypart.tolist(), 
    "home": home_or_away
})
features.head()

,distance_to_goal,angle_to_goal,bodypart,home
0,13.130000,55.958949,4,True
1,21.873601,16.247632,5,True
2,13.469135,20.695451,5,False
3,17.455569,38.557284,5,False
4,27.116462,14.523092,5,True


**Foram escolhidas tais features devido a hipótese de terem alto impacto na decisão se um chute foi gol ou não. Foi visto em aula que distância ao gol e ângulo ao mesmo são duas boas métricas que podem ser consideradas, com modelos já consolidados a utilizando. Além disso, parte do corpo utilizada e se o time é da casa ou não pode impactar bastante.**

## Questão 2
- Crie uma coluna numérica binária "goal" no dataframe "shots" indicando se o chute resultou em gol ou não.
- Use regressão logística [1] p/ treinar (.fit(X_train, y_train)) um modelo de xG usando as features criadas na questão 1.
- Use 70% dos dados para treino e 30% para teste.
- Reporte a acurácia do modelo para os conjuntos de treino (.score(X_train, y_train)) e teste (.score(X_test, y_test)).

In [103]:
shots.loc[:, "goal"] = (shots["result_name"] == "success").astype(int)

C:\Users\tonim\AppData\Local\Temp\ipykernel_10676\116450023.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0 0 0 ... 1 0 0]' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  shots.loc[:, "goal"] = (shots["result_name"] == "success").astype(int)


In [104]:
features["home"] = features["home"].astype(int)

In [105]:
features = pd.concat([features, pd.get_dummies(features["bodypart"]).astype(int)], axis=1).drop("bodypart", axis=1)

In [106]:
from sklearn.model_selection import train_test_split

X = features.values
y = shots["goal"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [107]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42).fit(X_train, y_train)

In [111]:
print("Acurácia de treino:", end="")
clf.score(X_train, y_train)

Acurácia de treino:

0.8950760966875559

In [112]:
print("Acurácia de teste:", end="")
clf.score(X_test, y_test)

Acurácia de teste:

0.8880534670008354

## Questão 3
- Use o modelo treinado na questão 2 p/ prever a probabilidade de gol de todos os chutes do dataframe "shots". Reporte essas probabilidades no dataframe "shots" em uma coluna "xG".
- Agrupe o dataframe "shots" por "player_name" e reporte a soma dos "goal" e "xG".
- Reporte os 10 jogadores com maior xG.
- Reporte os 10 jogadores com maior diferença de Gols e xG.

In [119]:
shots.loc[:, "xG"] = clf.predict_proba(X)[:, 1]

In [128]:
group = shots.groupby(["player_id", "player_name"]).sum()[["goal", "xG"]]
group

,,goal,xG
player_id,player_name,,
151,John Guidetti,3,2.943029
254,Roberto Jos\u00e9 Rosales Altuve,1,0.388475
786,R\u00f3bert Maz\u00e1\u0148,0,0.017389
1751,Oussama Tannane,0,0.914941
3269,Jordi Alba Ramos,2,2.334879
...,...,...,...
485383,Alejandro Miguel Mula Sanchez,0,1.190133
489417,Erik Alexander Exp\u00f3sito Hern\u00e1ndez,1,1.084543
489488,Mart\u00edn Aguirregabiria Padilla,0,0.158391


In [132]:
group.sort_values(by="xG", ascending=False).head(10)

,,goal,xG
player_id,player_name,,
3322,Cristiano Ronaldo dos Santos Aveiro,23,22.687072
7972,Luis Alberto Su\u00e1rez D\u00edaz,24,20.491104
3359,Lionel Andr\u00e9s Messi Cuccittini,26,19.457147
5400,Gerard Moreno Balaguero,15,14.585830
3840,Iago Aspas Juncal,19,12.972127
395636,Maximiliano G\u00f3mez Gonz\u00e1lez,18,12.839050
3714,Cristhian Ricardo Stuani Curbelo,16,10.669898
3874,Enrique Garc\u00eda Mart\u00ednez,8,9.937946
70129,Rodrigo Moreno Machado,15,9.917692


In [134]:
group["diff"] = group["goal"] - group["xG"]
group.sort_values(by="diff", ascending=False).head(10)

,,goal,xG,diff
player_id,player_name,,,
3682,Antoine Griezmann,16,7.998916,8.001084
3359,Lionel Andr\u00e9s Messi Cuccittini,26,19.457147,6.542853
8278,Gareth Frank Bale,15,8.890373,6.109627
3840,Iago Aspas Juncal,19,12.972127,6.027873
4131,\u00c1ngel Luis Rodr\u00edguez D\u00edaz,13,7.392240,5.607760
3714,Cristhian Ricardo Stuani Curbelo,16,10.669898,5.330102
395636,Maximiliano G\u00f3mez Gonz\u00e1lez,18,12.839050,5.160950
70129,Rodrigo Moreno Machado,15,9.917692,5.082308
355599,Mikel Oyarzabal Ugarte,12,7.044020,4.955980


## Questão 4
- Instancie um objeto ExpectedThreat [2] com parâmetros l=25 e w=16.
- Faça o fit do modelo ExpectedThreat com o dataframe "spadl".

In [136]:
import socceraction.xthreat as xthreat

xTModel = xthreat.ExpectedThreat(l=25, w=16)
xTModel.fit(spadl)

# iterations:  29


## Questão 5
- Crie um dataframe "prog_actions" à partir do dataframe "spadl", contendo apenas as ações de progressão e que são bem-sucedidas [3].
- Use o método rate do objeto ExpectedThreat p/ calcular o valor de cada ação de progressão do dataframe "prog_actions", em uma coluna chamada "action_value".
- Agrupe o dataframe "prog_actions" por "player_name" e reporte a soma dos "action_value".
- Reporte os 10 jogadores com maior "action_value".

In [138]:
prog_actions = xthreat.get_successful_move_actions(spadl)

In [140]:
prog_actions["action_value"] = xTModel.rate(prog_actions)

In [143]:
group_with_sum = prog_actions.groupby(["player_id", "player_name"]).sum()
group_with_sum["action_value"]

player_id  player_name                       
33         Jasper Cillessen                     -0.007225
99         Przemys\u0142aw Tyto\u0144            0.026717
151        John Guidetti                         1.552049
254        Roberto Jos\u00e9 Rosales Altuve      7.150794
786        R\u00f3bert Maz\u00e1\u0148           0.205801
                                                   ...   
519496     Fabio Gonz\u00e1lez Estupi\u00f1an    0.045368
520163     Diego Hern\u00e1ndez Barriuso         0.035621
545811     Hugo Duro Perales                    -0.019441
551398     David Alba Fern\u00e1ndez             0.023183
568583     Juan Cruz Diaz Esp\u00f3sito          0.008941
Name: action_value, Length: 556, dtype: float64

In [148]:
group_with_sum.sort_values(by="action_value", ascending=False).head(10)[["action_value"]]

,,action_value
player_id,player_name,
3310,Marcelo Vieira da Silva J\u00fanior,11.500223
3359,Lionel Andr\u00e9s Messi Cuccittini,10.782755
278289,\u00c1lvaro Odriozola Arzallus,8.911590
3325,Juan Francisco Moreno Fuertes,7.906469
3827,Hugo Mallo Novegil,7.895647
225089,Jos\u00e9 Luis Morales Nogales,7.886556
254,Roberto Jos\u00e9 Rosales Altuve,7.150794
3269,Jordi Alba Ramos,7.139224
3280,\u00c9ver Maximiliano David Banega,7.128678
